Excercise 5

Problem 2. Creating an interactive map showing areas in Helsinki from which Forum and Itis are accessable within 15 minutes by car during rush time 

In [1]:
import folium
from pyproj import crs
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Define file paths and read in the polygon grid for metropolitan area and travel times for Forum and Itis
fp = r'data/MetropAccess_YKR_grid_EurefFIN.shp'
grid = gpd.read_file(fp)

fp2 = r'data/TravelTimes_to_5975373_Forum.txt'
forum = pd.read_csv(fp2, sep=';')

fp3 = r'data/TravelTimes_to_5944003_Itis.txt'
itis = pd.read_csv(fp3, sep=';')

# Keep only useful columns of travel times data
forum = forum[['car_r_t','from_id']]
itis = itis[['car_r_t','from_id']]

# Rename 'from_id' to 'YKR_ID' to match with grid
forum  = forum.rename(columns={'from_id':'YKR_ID'})
itis  = itis.rename(columns={'from_id':'YKR_ID'})
print(forum.head(3))
print(itis.head(3))

   car_r_t   YKR_ID
0       49  5785640
1       49  5785641
2       57  5785642
   car_r_t   YKR_ID
0       50  5785640
1       51  5785641
2       58  5785642


In [2]:
# Join attributes from data to grid
forum_geo = grid.merge(forum, how='inner', on='YKR_ID')
forum_geo.head()

itis_geo = grid.merge(itis, how='inner', on='YKR_ID')
itis_geo.head()

,x,y,YKR_ID,geometry,car_r_t
0,381875.0,6697880.0,5785640,"POLYGON ((382000.000 6697750.000, 381750.000 6...",50
1,382125.0,6697880.0,5785641,"POLYGON ((382250.000 6697750.000, 382000.000 6...",51
2,382375.0,6697880.0,5785642,"POLYGON ((382500.000 6697750.000, 382250.000 6...",58
3,382625.0,6697880.0,5785643,"POLYGON ((382750.000 6697750.000, 382500.000 6...",57
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000 6697500.000, 381000.000 6...",50


In [3]:
# Remove NA values 
import numpy as np
# Remove no-data values (i.e. -1) from the data 
# First, replace -1 with 'NaN'
forum_geo = forum_geo.replace(-1, np.nan)
print(forum_geo.isna().sum())

itis_geo = itis_geo.replace(-1, np.nan)
print(itis_geo.isna().sum())

# Then, drop na rows
forum_geo = forum_geo.dropna()
itis_geo = itis_geo.dropna()

# Check there are no no-data values left in the data
print(forum_geo.isna().sum())
print(itis_geo.isna().sum())

x            0
y            0
YKR_ID       0
geometry     0
car_r_t     91
dtype: int64
x            0
y            0
YKR_ID       0
geometry     0
car_r_t     91
dtype: int64
x           0
y           0
YKR_ID      0
geometry    0
car_r_t     0
dtype: int64
x           0
y           0
YKR_ID      0
geometry    0
car_r_t     0
dtype: int64


In [5]:
# Choose grid cells that are within 15 min driving time
# Forum
f_geo_r = forum_geo.loc[forum_geo['car_r_t'] <= 15]
print(len(f_geo_r))

# Itis
i_geo_r = itis_geo.loc[itis_geo['car_r_t'] <= 15]
print(len(i_geo_r))


242
521


In [30]:
# Create a Map instance for interactive map
m = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=10, control_scale=True)

In [6]:
# Reproject travel times into WGS84
f_geo_r = f_geo_r.to_crs(epsg=4326)
i_geo_r = i_geo_r.to_crs(epsg=4326)

print(f_geo_r.crs)
print(i_geo_r.crs)


epsg:4326
epsg:4326


In [7]:
# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
f_geo_r['geoid'] = f_geo_r.index.astype(str)
i_geo_r['geoid'] = i_geo_r.index.astype(str)

In [9]:
f_geo_r = f_geo_r[['geoid','car_r_t','geometry']]
i_geo_r = i_geo_r[['geoid','car_r_t','geometry']]
i_geo_r.head()

,geoid,car_r_t,geometry
4641,4641,15.0,"POLYGON ((25.09630 60.28796, 25.09178 60.28789..."
4804,4804,15.0,"POLYGON ((25.09643 60.28571, 25.09191 60.28565..."
5465,5465,15.0,"POLYGON ((25.08340 60.27655, 25.07888 60.27648..."
5467,5467,15.0,"POLYGON ((25.09243 60.27668, 25.08792 60.27661..."
5631,5631,15.0,"POLYGON ((25.08353 60.27430, 25.07901 60.27424..."


In [14]:
# Add shopping center locations
# Read in shopping center locations
fp4 = r'data/shopping_centers.shp'

SCs= gpd.read_file(fp4)
print(SCs.crs)
SCs.head()

SCs = SCs.to_crs(epsg=4326)
print(SCs.crs)

epsg:3879
epsg:4326


In [20]:
# Take only Forum and Itis
forum_itis = SCs.loc[(SCs['name'] == 'Forum') | (SCs['name'] == 'Itis')]
forum_itis

# Convert points to GeoJSON
fi_points_gjson = folium.features.GeoJson(forum_itis, name="Shopping centers")

In [33]:
folium.Choropleth(
    geo_data=f_geo_r,
    name='Travel times by car',
    data=f_geo_r,
    columns=['geoid', 'car_r_t'],
    key_on='feature.id',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Travel time (min) to Forum by car').add_to(m)

folium.Choropleth(
    geo_data=i_geo_r,
    name='Travel times by car',
    data=i_geo_r,
    columns=['geoid', 'car_r_t'],
    key_on='feature.id',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Travel time (min) to Itis by car').add_to(m)

# Add points to the map instance
fi_points_gjson.add_to(m)

#Show map
m